### Imports

#### Import Packages

In [1]:
import polars as pl
import altair as alt
import numpy as np
import great_tables as GT
import gc

alt.JupyterChart.enable_offline()
alt.renderers.enable("jupyter", offline=True)

RendererRegistry.enable('jupyter')

#### Import Panel Data

In [2]:
grocery_lf = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv",
                         has_header=True,
                         separator=",",
                         schema_overrides={'panel_id': pl.Int32,
                                           'trans_id': pl.Int32,
                                           'week': pl.Int16,
                                           'sku_id': pl.Categorical,
                                           'units': pl.Int16,
                                           'price': pl.Float32,
                                           'brand': pl.Categorical})
grocery_lf.head().collect()

panel_id,trans_id,week,sku_id,units,price,brand
i32,i32,i16,cat,i16,f32,cat
3102011,1569,6,"""19""",1,2.79,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""38""",1,3.49,"""Bravo"""
3102012,4301,15,"""44""",1,3.49,"""Bravo"""


In [3]:
sku_lf = pl.scan_csv(source="data/panel-datasets/sku_weight.csv",
                         has_header=True,
                         separator=",",
                         schema_overrides={'sku_id': pl.Int32,
                                           'weight': pl.Int16})
sku_lf.head().collect()

sku_id,weight
i32,i16
1,400
2,400
3,400
4,250
5,1000


In [4]:
kiwi_lf = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv",
                      has_header=True,
                      separator=",",
                      schema_overrides={'ID': pl.Int16,
                                        'Market': pl.Categorical,
                                        'Day': pl.Int16,
                                        'Units': pl.Int16})
kiwi_lf.head().collect()

ID,Market,Week,Day,Units
i16,cat,i64,i16,i16
10001,"""1""",19,3,1
10002,"""1""",12,5,1
10003,"""1""",37,7,1
10004,"""1""",30,6,1
10004,"""1""",47,3,1


### Preliminaries

#### Weekly Sales Pattern

In [5]:
# Grocery LazyFrame (Query Plan) with New 'spend' Column - units x price = spend
grocery_lf_spend = grocery_lf.with_columns(
    ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
)

In [6]:
# Category Weekly Sales - LazyFrame (Query Plan)
weekly_sales_lf = (
    grocery_lf_spend
    .group_by(["week"])
    .agg(
        pl.col("spend").sum().cast(pl.Float64).alias('Weekly Spend')
    )
    .sort("week")   
)

# Weekly Sales by Brand - LazyFrame (Query Plan)
weekly_sales_lf_brand = (
    grocery_lf_spend
    .group_by(["week", "brand"])
    .agg(
        pl.col("spend").sum().alias('Weekly Spend')
    )
    .sort("week")   
)

In [7]:
# Weekly Sales Pivot Table - Polars DataFrame
# For visualizing & inspecting the table only
weekly_sales_pivot = weekly_sales_lf_brand.collect().pivot(
    on="brand",
    index="week",
    values="Weekly Spend",
    sort_columns=True,
).sort("week").with_columns(
    pl.sum_horizontal(pl.all()).alias("Total") # Row total
)

weekly_sales_pivot

week,Alpha,Bravo,Charlie,Delta,Other,Total
i16,f64,f64,f64,f64,f64,f64
1,331.459999,247.780003,51.79,17.64,18.75,668.420002
2,567.249997,398.640007,45.43,34.19,23.23,1070.740004
3,497.969999,639.980008,51.11,39.39,14.55,1246.000006
4,1389.960004,472.500004,51.87,85.04,39.779999,2043.150007
5,358.239999,252.540003,40.21,69.54,45.93,771.460002
…,…,…,…,…,…,…
100,692.480006,420.479996,69.600001,54.590001,47.97,1385.120004
101,265.770002,264.659999,64.030001,35.05,26.88,757.390003
102,379.610005,456.78999,43.5,20.4,43.61,1045.909995


In [8]:
chart = alt.Chart(weekly_sales_pivot).mark_line().encode(
    x=alt.X('week', axis=alt.Axis(
        values=np.arange(0, 104+1, 13),  # Explicitly specify quarter-end weeks
        labelExpr="datum.value",  # Show only these labels
        title='Week')),
    y=alt.Y('Alpha',
            title='Spend ($)')
).properties(
    width=900,
    height=300
)

jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…

In [9]:
chart = alt.Chart(weekly_sales_pivot).mark_line().encode(
    x=alt.X('week', axis=alt.Axis(
        values=np.arange(0, 104+1, 13),  # Explicitly specify quarter-end weeks
        labelExpr="datum.value",  # Show only these labels
        title='Week')),
    y=alt.Y('Total',
            title='Spend ($)')
).properties(
    width=900,
    height=300
)

jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…